In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self, node1=120, node2=84):
        super().__init__() 
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, node1)
        
        self.fc2 = nn.Linear(node1, node2)
        self.fc3 = nn.Linear(node2, 10)
        
    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool1(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

/Users/panda/Personal/PyTorch-All-In-1/venv/lib/python3.10/site-packages/torch/nn/modules/transformer.py:20: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_numpy.cpp:84.)
  device: torch.device = torch.device(torch._C._get_default_device()),  # torch.device('cpu'),


In [ ]:
from ray import tune
import numpy as np

config = {
    "node1": tune.sample_from(lambda _: 2 ** np.random.randint(2,9)),
    "node2": tune.sample_from(lambda _: 2 ** np.random.randint(2,9)),
    "lr": tune.loguniform(1e-4, 1e-1),
    "batch_size": tune.choice([2, 4, 8, 16])
}

In [ ]:
import torchvision
import torchvision.transforms as transforms

def load_data(data_dir="./data", batch_size=4):
    transform = transforms.Compose(
        [transforms.ToTensor(),
         transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

    trainset = torchvision.datasets.CIFAR10(root=data_dir, train=True,
                                            download=True, transform=transform)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                              shuffle=True, num_workers=2)

    testset = torchvision.datasets.CIFAR10(root=data_dir, train=False,
                                           download=True, transform=transform)
    testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                             shuffle=False, num_workers=2)
    return trainloader, testloader

In [ ]:
import torch.optim as optim
from torch import nn
from torch.utils.data import random_split

def train_cifar(config):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    
    model=Net(config["node1"], config["node2"]).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=config["lr"], momentum=0.9)
    
    train_data, test_data = load_data(batch_size=config["batch_size"])
    
    for epoch in range(10):
        running_loss = 0.0
        for i, data in enumerate(train_data, 0):
            inputs, labels = data[0].to(device), data[1].to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            if i % 2000 == 1999:
                print(f"[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}")
                running_loss = 0.0
        
        for i, data in enumerate(test_data, 0):
            inputs, labels = data[0].to(device), data[1].to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item()
        print(f"Test loss: {running_loss / len(test_data):.3f}")
        
    return running_loss / len(test_data)
    

In [ ]:
from sched import scheduler
from ray.tune.schedulers import ASHAScheduler
from ray.tune import CLIReporter

scheduler = ASHAScheduler(
    metric="loss",
    mode='min',
    max_t=10,
    grace_period=1,
    reduction_factor=2
)

reporter = CLIReporter(
    metric_columns=['loss','accuracy','training_iteration']
)

In [ ]:
from functools import partial


reuslt=tune.run(
    partial(train_cifar),
    resources_per_trial={"cpu":2, "gpu":1},
    config=config,
    num_samples=10,
    scheduler=scheduler,
    progress_reporter=reporter
)

In [ ]:
best_trial = result.get_best_trial(
    "loss", "min", "last")
print("Best trial config: {}".format(
    best_trial.config))
print("Best trial final validation loss:",
      "{}".format(
          best_trial.last_result["loss"]))
print("Best trial final validation accuracy:",
      "{}".format(
          best_trial.last_result["accuracy"]))